In [1]:
import sys.process._

In [2]:
//"hdfs dfs -ls /user".!

Name: Syntax Error.
Message: 
StackTrace: 

In [3]:
//"hdfs dfs -ls /tmp/".!

Name: Syntax Error.
Message: 
StackTrace: 

## Cassandra

In [4]:
%AddJar https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.11/2.4.3/spark-cassandra-connector_2.11-2.4.3.jar

Using cached version of spark-cassandra-connector_2.11-2.4.3.jar


## Elasticsearch

In [5]:
%AddJar file:///data/home/alexey.chernyaev2/elasticsearch-spark-20_2.11-7.7.0.jar

Using cached version of elasticsearch-spark-20_2.11-7.7.0.jar


## Postgres

In [6]:
%AddJar file:///data/home/alexey.chernyaev2/postgresql-42.2.12.jre6.jar

Using cached version of postgresql-42.2.12.jre6.jar


In [7]:
import org.apache.spark.{SparkContext, SparkConf}

println("Starting Spark Session...")

val username = "alexey_chernyaev2"
val password = "fuiBpd2S"
val hostname = "10.0.0.5"

val postgreUrl = s"jdbc:postgresql://$hostname:5432"

val conf = new SparkConf()
                    .set("spark.cassandra.connection.host", hostname)
                    .set("spark.cassandra.connection.port", "9042")
                    .set("spark.cassandra.auth.username", "alexey.chernyaev2")
                    .set("spark.cassandra.auth.password", password)
                    .set("spark.cassandra.output.consistency.level", "ANY")
                    .set("spark.cassandra.input.consistency.level", "ONE")
    
val sparkSession = SparkSession.builder()
                        .appName("Lab03")
                        .config(conf=conf)
                        .getOrCreate()

println("Spark Session was started")

Starting Spark Session...
Spark Session was started


username = alexey_chernyaev2
password = fuiBpd2S
hostname = 10.0.0.5
postgreUrl = jdbc:postgresql://10.0.0.5:5432
conf = org.apache.spark.SparkConf@581191e
sparkSession = org.apache.spark.sql.SparkSession@374494b6


org.apache.spark.sql.SparkSession@374494b6

In [8]:
// read cassandra 

val tableOpts = Map("table" -> "clients", "keyspace" -> "labdata")
val clients = spark
  .read
  .format("org.apache.spark.sql.cassandra")
  .options(tableOpts)
  .load()
clients.show()

+--------------------+---+------+
|                 uid|age|gender|
+--------------------+---+------+
|075bdfc4-cfa6-4f0...| 39|     F|
|a1096800-f5d8-4f1...| 52|     M|
|594d59f5-22db-4c2...| 39|     M|
|e60a27f7-9c8e-4c6...| 30|     M|
|bf620165-604f-4ea...| 65|     F|
|97fd1ca4-b2b6-4f8...| 20|     F|
|d219a21c-af6d-484...| 39|     M|
|0e38d915-ed29-4ab...| 49|     M|
|9f968029-11c0-429...| 39|     F|
|1784f7ee-f8fa-419...| 19|     F|
|400a54aa-51dd-4e3...| 32|     F|
|1ffe78da-81b1-4ea...| 39|     M|
|58fd46de-2e9d-4b8...| 26|     M|
|d066f4cb-fa3f-4a9...| 30|     F|
|8608a863-1abe-47f...| 26|     M|
|a4404d1c-9d82-451...| 31|     F|
|d226a5bf-287d-47c...| 36|     M|
|2d256dd9-771d-4e8...| 29|     M|
|b9ad76d6-b20f-4e5...| 34|     F|
|eee0b882-a6a8-4f3...| 25|     M|
+--------------------+---+------+
only showing top 20 rows



tableOpts = Map(table -> clients, keyspace -> labdata)
clients = [uid: string, age: int ... 1 more field]


[uid: string, age: int ... 1 more field]

In [9]:
import org.apache.spark.sql.functions.when

val shopStatsDF = clients
                        .withColumn("age_cat", when($"age" >= 55, ">=55")
                                              .when($"age" >= 45, "45-54")
                                              .when($"age" >= 35, "35-44")
                                              .when($"age" >= 25, "25-34")
                                              .when($"age" >= 18, "18-24"))
                        .join(visitStatsDF, Seq("uid"), "left")
                        .groupBy($"uid", $"gender", $"age_cat")
                        .pivot($"shop_group")
                        .sum("count")
                        .drop($"null")

shopStatsDF.show(5)

+--------------------+------+-------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+------------------+----------+---------------------+---------+
|                 uid|gender|age_cat|shop_cameras|shop_clothing|shop_computers|shop_cosmetics|shop_entertainment_equipment|shop_everyday_jewelry|shop_house_repairs_paint_tools|shop_household_appliances|shop_household_furniture|shop_kitchen_appliances|shop_kitchen_utensils|shop_luggage|shop_mobile_phones|shop_shoes|shop_sports_equipment|shop_toys|
+--------------------+------+-------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+----------------

shopStatsDF = [uid: string, gender: string ... 17 more fields]


[uid: string, gender: string ... 17 more fields]

In [10]:
// read HDFS

val webLogsDF = spark.read.parquet("/labs/laba03/weblogs.parquet")

webLogsDF.printSchema
webLogsDF.show(3, 200, true)

root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- url: string (nullable = true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 uid    | 39d8aded-2111-42ab-9393-37c4f781d030                                                                                                                                                                     
 visits | [[1419590902274, http://go.mail.ru/], [1419590344372, http://thephilanews.com/ru/wawa-welcome-america-begins-with-reading-…-and-the-runway-32987.htm], [1419590203997, http://visitorsworth.com/www.w... 
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------------

webLogsDF = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [11]:
import org.apache.spark.sql.functions._

val webLogsStatsDF = webLogsDF
                        .select($"uid", explode($"visits").as("visit"))
                        .withColumn("host", lower(callUDF("parse_url", $"visit.url", lit("HOST"))))
                        .withColumn("domain", regexp_replace(lower($"host"), "www.", ""))
                        .where($"uid".isNotNull)
                        .groupBy($"uid", $"domain").count

println(webLogsStatsDF.cache.count)
webLogsStatsDF.printSchema
webLogsStatsDF.show(5)

572694
root
 |-- uid: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- count: long (nullable = false)

+--------------------+----------------+-----+
|                 uid|          domain|count|
+--------------------+----------------+-----+
|3a081c72-5c55-4c7...|videograbber.net|    1|
|3a48922d-4621-455...|      umeltsi.ru|    4|
|63046a3d-830e-44b...|      intvua.com|    2|
|6305c993-f1e7-4fc...|     parasola.ru|    1|
|6307cb7e-2b98-4be...|  kontaktlife.ru|    1|
+--------------------+----------------+-----+
only showing top 5 rows



webLogsStatsDF = [uid: string, domain: string ... 1 more field]


[uid: string, domain: string ... 1 more field]

In [12]:
// read postrgres

val domainCatsDF = spark.read
                                .format("jdbc")
                                .option("driver", "org.postgresql.Driver")
                                .option("url", s"$postgreUrl/labdata")
                                .option("dbtable", "domain_cats")
                                .option("user", username)
                                .option("password", password)
                                .load()

domainCatsDF.printSchema
domainCatsDF.show()

root
 |-- domain: string (nullable = true)
 |-- category: string (nullable = true)

+--------------------+--------+
|              domain|category|
+--------------------+--------+
|national-lottery....|gambling|
|          rivalo.com|gambling|
|             xoso.me|gambling|
|         betin.co.ke|gambling|
|          skybet.com|gambling|
|              fdj.fr|gambling|
|     minhngoc.net.vn|gambling|
|   hollywoodbets.net|gambling|
|           tipico.de|gambling|
|       betpawa.co.ke|gambling|
|           bet365.it|gambling|
|          bet365.com|gambling|
|loteriasdominican...|gambling|
|            hkjc.com|gambling|
|        sunmaker.com|gambling|
|     williamhill.com|gambling|
|       parimatch.com|gambling|
|        freebitco.in|gambling|
|   mismarcadores.com|gambling|
|      flashscore.com|gambling|
+--------------------+--------+
only showing top 20 rows



domainCatsDF = [domain: string, category: string]


[domain: string, category: string]

In [13]:
val webStatsDF = webLogsStatsDF
                        .join(domainCatsDF, Seq("domain"), "left")
                        .withColumn("web_cat", concat(lit("web_"), regexp_replace(lower($"category"), "[ -]", "_")))
                        .where($"web_cat".isNotNull)
                        .groupBy($"uid")
                        .pivot($"web_cat")
                        .sum("count")


webStatsDF.show(5)

+--------------------+--------------------------+----------------------+----------------------+------------------------+-------------------------+------------------------+----------------------------+-----------+------------------+------------+---------+----------+-------------------+------------------------+----------------------+------------------+--------------------+--------------------------+-------------+-----------+------------+----------+----------+
|                 uid|web_arts_and_entertainment|web_autos_and_vehicles|web_beauty_and_fitness|web_books_and_literature|web_business_and_industry|web_career_and_education|web_computer_and_electronics|web_finance|web_food_and_drink|web_gambling|web_games|web_health|web_home_and_garden|web_internet_and_telecom|web_law_and_government|web_news_and_media|web_pets_and_animals|web_recreation_and_hobbies|web_reference|web_science|web_shopping|web_sports|web_travel|
+--------------------+--------------------------+----------------------+----

webStatsDF = [uid: string, web_arts_and_entertainment: bigint ... 22 more fields]


[uid: string, web_arts_and_entertainment: bigint ... 22 more fields]

In [14]:
// read elasticsearch

val esOptions =
  Map(
    "es.nodes" -> "10.0.0.5:9200",
    "es.batch.write.refresh" -> "false",
    "es.nodes.wan.only" -> "true",
    "es.net.http.auth.user" -> "alexey.chernyaev2",
    "es.net.http.auth.pass" -> password
  )
var visitsDF = spark.read.format("org.elasticsearch.spark.sql").options(esOptions).load("visits*")


visitsDF.show(5)
visitsDF.printSchema

+--------------------+----------+--------------------+----------+-------------------+--------------------+
|            category|event_type|             item_id|item_price|          timestamp|                 uid|
+--------------------+----------+--------------------+----------+-------------------+--------------------+
|Entertainment-equ...|       buy|Entertainment-equ...|      2529|2020-01-01 11:00:00|40b29579-e845-45c...|
|    Everyday-jewelry|       buy|  Everyday-jewelry-0|      4320|2020-01-01 11:01:00|                null|
|             Cameras|      view|           Cameras-1|      1856|2020-01-01 11:02:00|ab0e7dd1-5899-488...|
|             Luggage|      view|           Luggage-7|      4975|2020-01-01 11:02:00|                null|
|       Mobile-phones|      view|    Mobile-phones-10|      3981|2020-01-01 11:03:00|d1206327-0e9f-410...|
+--------------------+----------+--------------------+----------+-------------------+--------------------+
only showing top 5 rows

root
 |-- ca

esOptions = Map(es.nodes.wan.only -> true, es.net.http.auth.user -> alexey.chernyaev2, es.net.http.auth.pass -> fuiBpd2S, es.batch.write.refresh -> false, es.nodes -> 10.0.0.5:9200)
visitsDF = [category: string, event_type: string ... 4 more fields]


[category: string, event_type: string ... 4 more fields]

In [15]:
val visitStatsDF = visitsDF
                    .filter($"uid".isNotNull)
                    .withColumn("shop_group", concat(lit("shop_"), regexp_replace(lower($"category"), "[ -]", "_")))
                    .groupBy($"uid", $"shop_group")
                    .count()

visitStatsDF.show(5)

+--------------------+--------------------+-----+
|                 uid|          shop_group|count|
+--------------------+--------------------+-----+
|75fcbe0c-ef8a-452...|      shop_computers|    1|
|9e241ecb-af15-49a...|        shop_cameras|    1|
|372c086a-c53d-45f...|  shop_mobile_phones|    1|
|fcd5dccb-6efc-447...|shop_entertainmen...|    1|
|b065187e-5ab9-46c...|        shop_luggage|    1|
+--------------------+--------------------+-----+
only showing top 5 rows



visitStatsDF = [uid: string, shop_group: string ... 1 more field]


[uid: string, shop_group: string ... 1 more field]

## DataMart

In [16]:
val datamartDF = shopStatsDF
                        .join(webStatsDF, Seq("uid"), "left")
                        .orderBy($"uid")
println(datamartDF.cache.count)
datamartDF.printSchema
datamartDF.show(3, 200, true)

36138
root
 |-- uid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_cat: string (nullable = true)
 |-- shop_cameras: long (nullable = true)
 |-- shop_clothing: long (nullable = true)
 |-- shop_computers: long (nullable = true)
 |-- shop_cosmetics: long (nullable = true)
 |-- shop_entertainment_equipment: long (nullable = true)
 |-- shop_everyday_jewelry: long (nullable = true)
 |-- shop_house_repairs_paint_tools: long (nullable = true)
 |-- shop_household_appliances: long (nullable = true)
 |-- shop_household_furniture: long (nullable = true)
 |-- shop_kitchen_appliances: long (nullable = true)
 |-- shop_kitchen_utensils: long (nullable = true)
 |-- shop_luggage: long (nullable = true)
 |-- shop_mobile_phones: long (nullable = true)
 |-- shop_shoes: long (nullable = true)
 |-- shop_sports_equipment: long (nullable = true)
 |-- shop_toys: long (nullable = true)
 |-- web_arts_and_entertainment: long (nullable = true)
 |-- web_autos_and_vehicles: long (nullable = 

datamartDF = [uid: string, gender: string ... 40 more fields]


[uid: string, gender: string ... 40 more fields]

In [17]:
val datamartDF = shopStatsDF
                        .join(webStatsDF, Seq("uid"), "left")
                        .orderBy($"uid")
println(datamartDF.cache.count)
datamartDF.printSchema
datamartDF.show(5)

36138
root
 |-- uid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_cat: string (nullable = true)
 |-- shop_cameras: long (nullable = true)
 |-- shop_clothing: long (nullable = true)
 |-- shop_computers: long (nullable = true)
 |-- shop_cosmetics: long (nullable = true)
 |-- shop_entertainment_equipment: long (nullable = true)
 |-- shop_everyday_jewelry: long (nullable = true)
 |-- shop_house_repairs_paint_tools: long (nullable = true)
 |-- shop_household_appliances: long (nullable = true)
 |-- shop_household_furniture: long (nullable = true)
 |-- shop_kitchen_appliances: long (nullable = true)
 |-- shop_kitchen_utensils: long (nullable = true)
 |-- shop_luggage: long (nullable = true)
 |-- shop_mobile_phones: long (nullable = true)
 |-- shop_shoes: long (nullable = true)
 |-- shop_sports_equipment: long (nullable = true)
 |-- shop_toys: long (nullable = true)
 |-- web_arts_and_entertainment: long (nullable = true)
 |-- web_autos_and_vehicles: long (nullable = 

datamartDF = [uid: string, gender: string ... 40 more fields]


[uid: string, gender: string ... 40 more fields]

## Save

In [18]:
val options = Map(
                 "url" -> s"$postgreUrl/$username",
                 "user" -> username,
                 "password" -> password,
                 "mode" -> "overwrite",
                 "driver" -> "org.postgresql.Driver",
                 "dbtable" -> "clients")

datamartDF.write.format("jdbc").options(options).mode("overwrite").save()

options = Map(url -> jdbc:postgresql://10.0.0.5:5432/alexey_chernyaev2, driver -> org.postgresql.Driver, dbtable -> clients, mode -> overwrite, user -> alexey_chernyaev2, password -> fuiBpd2S)


Map(url -> jdbc:postgresql://10.0.0.5:5432/alexey_chernyaev2, driver -> org.postgresql.Driver, dbtable -> clients, mode -> overwrite, user -> alexey_chernyaev2, password -> fuiBpd2S)